In [2]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds

In [3]:
data_dir = '../raw_data'

In [5]:
ds = tf.keras.utils.image_dataset_from_directory(
  data_dir)

Found 55448 files belonging to 39 classes.


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 10))
for images, labels in ds.take(2):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(ds.class_names[labels[i]])
        plt.axis("off")

In [ ]:
for image_batch, labels_batch in ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

In [ ]:
classes = ds.class_names

In [26]:
classes = ds.class_names
numbers= np.arange(1,40,1)
dict_leaves = {classes[i]: numbers[i] for i in range(len(classes))}
dict_leaves

{'Apple___Apple_scab': 1,
 'Apple___Black_rot': 2,
 'Apple___Cedar_apple_rust': 3,
 'Apple___healthy': 4,
 'Background_without_leaves': 5,
 'Blueberry___healthy': 6,
 'Cherry___Powdery_mildew': 7,
 'Cherry___healthy': 8,
 'Corn___Cercospora_leaf_spot Gray_leaf_spot': 9,
 'Corn___Common_rust': 10,
 'Corn___Northern_Leaf_Blight': 11,
 'Corn___healthy': 12,
 'Grape___Black_rot': 13,
 'Grape___Esca_(Black_Measles)': 14,
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)': 15,
 'Grape___healthy': 16,
 'Orange___Haunglongbing_(Citrus_greening)': 17,
 'Peach___Bacterial_spot': 18,
 'Peach___healthy': 19,
 'Pepper,_bell___Bacterial_spot': 20,
 'Pepper,_bell___healthy': 21,
 'Potato___Early_blight': 22,
 'Potato___Late_blight': 23,
 'Potato___healthy': 24,
 'Raspberry___healthy': 25,
 'Soybean___healthy': 26,
 'Squash___Powdery_mildew': 27,
 'Strawberry___Leaf_scorch': 28,
 'Strawberry___healthy': 29,
 'Tomato___Bacterial_spot': 30,
 'Tomato___Early_blight': 31,
 'Tomato___Late_blight': 32,
 'Tomato_

In [ ]:
ds.element_spec

In [ ]:
# y= ds.class_names
# X= 

In [ ]:
sample = next(iter(ds))
sample

In [ ]:
#tf.keras.layers.Rescaling(1./255)


In [33]:
train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    labels='inferred',
    #label_mode=None,
    #class_names="class_names",
    color_mode='rgb',
    batch_size=32,
    image_size=(256, 256),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset="both")  

Found 55448 files belonging to 39 classes.
Using 44359 files for training.
Using 11089 files for validation.


In [ ]:
# train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(
#     'temp',
#     labels='inferred',
#     #label_mode=None,
#     #class_names="class_names",
#     color_mode='rgb',
#     batch_size=32,
#     image_size=(256, 256),
#     shuffle=True,
#     seed=123,
#     validation_split=0.1,
#     subset="both") 

In [34]:
len(ds.class_names)

39

In [ ]:
# val_ds = tf.keras.utils.image_dataset_from_directory(
#     data_dir,
#     labels='inferred',
#     color_mode='rgb',
#     batch_size=32,
#     image_size=(256, 256),
#     shuffle=False,
#     seed=123,
#     validation_split=0.2,
#     subset="validation")

In [35]:
train_ds

<BatchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [ ]:
# ds.as_numpy_iterator().next()

In [ ]:
# Split the dataset into training, test, and validation sets
# temp = ds
# test_dataset = tf.keras.utils.image_dataset_from_directory(
#     data_dir,
#     image_size=(256, 256),
#     batch_size=32,
#     validation_split=0.2,
#     subset="validation",
#     seed=42
# )

/

In [ ]:
# train_to_numpy = list(train_dataset.as_numpy_iterator())
# shape = tf.shape(dataset_to_numpy)
# print(shape)

In [36]:
from tensorflow.keras import models, losses
from tensorflow.keras import Sequential, layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Rescaling, RandomFlip, RandomRotation

model = models.Sequential()

model.add(layers.Rescaling(1./255))
model.add(layers.RandomFlip("horizontal_and_vertical"))
model.add(layers.RandomRotation(0.2))
model.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(256, 256, 3)))
model.add(layers.MaxPool2D(pool_size=(2,2))) 
model.add(layers.Conv2D(32, kernel_size=(3), activation='relu')) # kernel_size = 3 <==> (3, 3)
model.add(layers.MaxPool2D(pool_size=(2,2))) 
model.add(layers.Flatten())
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dense(39, activation='softmax')) 

#model.summary() 

In [ ]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              optimizer='adam',
              metrics=['accuracy'])

model.fit(train_ds,
          validation_data=val_ds,
          epochs=2,  # Use early stopping in practice
          batch_size=32, 
          verbose=1)

In [ ]:
model.summary()

In [44]:
from tensorflow.keras.applications import vgg16
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers


# Loading the weights of a pre-trained ConvNet
pretrained = vgg16.VGG16(weights='imagenet', 
                         include_top=False,
                         input_shape=(256, 256, 3))

pretrained.trainable = False
# Optional, if you have time and want to re-train these weights, you can set it to True.

# Flatten + Intermediate Dense Layer + Predictive Layer
flatten_layer = layers.Flatten()
dense_layer = layers.Dense(100, activation='relu')
prediction_layer = layers.Dense(38, activation='softmax')

model1 = Sequential([
    pretrained, 
#     tf.keras.layers.Rescaling(1./255, input_shape=(256,256,3)),
#     tf.keras.layers.RandomFlip("horizontal_and_vertical"),
#     tf.keras.layers.CenterCrop(height=224, width=224), 
#     tf.keras.layers.RandomRotation(0.2),
#     tf.keras.layers.Conv2D(64, 4, activation='relu'),
#     tf.keras.layers.MaxPooling2D(),
#     tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(246, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(39, activation='softmax')
])

In [45]:
model1.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 8, 8, 512)         14714688  
                                                                 
 flatten_11 (Flatten)        (None, 32768)             0         
                                                                 
 dense_26 (Dense)            (None, 246)               8061174   
                                                                 
 dropout_8 (Dropout)         (None, 246)               0         
                                                                 
 dense_27 (Dense)            (None, 128)               31616     
                                                                 
 dropout_9 (Dropout)         (None, 128)               0         
                                                                 
 dense_28 (Dense)            (None, 39)               

In [46]:
model1.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              optimizer='adam',
              metrics=['accuracy'])

model1.fit(train_ds,
          validation_data=val_ds,
          epochs=1,  # Use early stopping in practice
          batch_size=32, 
          verbose=1)

   7/1387 [..............................] - ETA: 1:49:20 - loss: 27.8989 - accuracy: 0.0848

KeyboardInterrupt: 

In [15]:
import os
print(os.getcwd())

/home/alice/code/Solid32/LeafScan-back/notebooks


In [61]:
import csv
result={}

# #Load the CSV file
# with open('../leafscan/diseases.csv', 'r') as data:
#     for line in csv.DictReader(data):
#         print(line)



def disease_info(disease_name):
    '''
    return corresponding data for a given disease
    '''
    result['plant_name'] = disease_name.split("__")[0]
    result['disease_name'] = disease_name.split("__")[1].replace("_", " ")
    return result

disease_name = 'Apple___Cedar_apple_rust'
disease_info(disease_name)

{'plant_name': 'Apple', 'disease_name': ' Cedar apple rust'}

In [64]:
def create_dictionary_from_csv(csv_file):
    dictionary = {}
    with open(csv_file, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            # Assumant que la première colonne contient les clés et la deuxième colonne les valeurs
            key = disease_name
            value = url
            dictionary[key] = value
    return dictionary

csv_file = '../leafscan/diseases.csv'
create_dictionary_from_csv(csv_file)

{'Apple___Cedar_apple_rust': None}

In [66]:
def create_dictionary_from_csv(csv_file):
    dictionary = {}
    with open(csv_file, 'r') as file:
        reader = csv.DictReader(file, delimiter=';')
        for row in reader:
            key = row['disease_name']
            value = row['url']
            dictionary[key] = value
    return dictionary

csv_file = '../leafscan/diseases.csv'
my_dictionary = create_dictionary_from_csv(csv_file)

my_dictionary

{'Apple___Apple_scab ': ' https://www.planetnatural.com/pest-problem-solver/plant-disease/apple-scab/',
 'Apple___Black_rot ': ' https://extension.umn.edu/plant-diseases/black-rot-apple',
 'Apple___Cedar_apple_rust ': ' https://gardenerspath.com/how-to/disease-and-pests/cedar-apple-rust-control/',
 'Apple___healthy ': '',
 'Background_without_leaves ': '',
 'Blueberry___healthy ': '',
 'Cherry___Powdery_mildew ': ' https://treefruit.wsu.edu/crop-protection/disease-management/cherry-powdery-mildew/',
 'Cherry___healthy': '',
 "'Corn___Cercospora_leaf_spot Gray_leaf_spot' ": ' https://en.wikipedia.org/wiki/Corn_grey_leaf_spot',
 'Corn___Common_rust ': 'https://extension.umn.edu/corn-pest-management/common-rust-corn#:~:text=Common%20rust%20produces%20rust%2Dcolored,as%20sheaths%2C%20can%20be%20infected.',
 'Corn___Northern_Leaf_Blight ': ' https://extension.umn.edu/corn-pest-management/northern-corn-leaf-blight',
 'Grape___Black_rot ': 'https://ohioline.osu.edu/factsheet/plpath-fru-24',
 

In [2]:
import csv
result={}

#Load the CSV file
with open('../leafscan/diseases.txt', 'r') as data:
    csvreader = csv.reader(data, delimiter=';')
    d = {rows[0].strip():rows[1].strip() for rows in csvreader}

def disease_info(disease_name):
    '''
    return corresponding data for a given disease
    '''
    result['plant_name'] = disease_name.split("__")[0]
    result['disease_name'] = disease_name.split("__")[1].replace("_", " ")
    result['url']=d[disease_name.strip()]
    return result

print(disease_info('Apple___Cedar_apple_rust'))

{'plant_name': 'Apple', 'disease_name': ' Cedar apple rust', 'url': 'https://gardenerspath.com/how-to/disease-and-pests/cedar-apple-rust-control/'}
